## Mask amygdala hippocampus and correlate the trauma script

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import nilearn.plotting
import nilearn.input_data
import os
import nipype.pipeline.engine as pe  # pypeline engine
import nipype.interfaces.utility as util  # utility

In [ ]:
output_dir = '/media/Data/work/kpe_connAnalysis'
ses = '1'

In [ ]:
## set parameters for the maskers
mask_params = {
               'detrend': True, 'standardize': True,
               'high_pass': 0.01, 'low_pass': 0.1, 't_r': 1,
               'smoothing_fwhm': 4,
                'verbose': 5}



In [ ]:
## Amygdala as mask
amg_file = '/media/Data/work/KPE_ROI/amygdala_association-test_z_FDR_0.01.nii.gz'
amg_file = nilearn.image.math_img("a>=25", a=amg_file)
%matplotlib inline
nilearn.plotting.plot_roi(amg_file)


masker_amg = nilearn.input_data.NiftiMasker(mask_img= amg_file, **mask_params).fit()

In [ ]:
# now lets do the same with vmPFC
vmpfc_mask = '/media/Data/work/RCF_or/vmpfc_association-test_z_FDR_0.01.nii.gz'
vmpfc_mask = nilearn.image.math_img("a>=5", a=vmpfc_mask)
%matplotlib inline
nilearn.plotting.plot_roi(vmpfc_mask)
masker_vmpfc = nilearn.input_data.NiftiMasker(mask_img= vmpfc_mask, **mask_params).fit()

In [ ]:
## Hippocampus
hipp_mask = '/media/Data/work/KPE_ROI/hippocampus_association-test_z_FDR_0.01.nii.gz'
hipp_mask = nilearn.image.math_img("a>=15", a=hipp_mask)
%matplotlib inline
nilearn.plotting.plot_roi(hipp_mask)
masker_hipp = nilearn.input_data.NiftiMasker(mask_img= mask_file, **mask_params).fit()

In [ ]:
def removeVars (sub, ses):
    
    # this method takes the csv regressors file (from fmriPrep) and chooses a few to confound. You can change those few
    import pandas as pd
    import numpy as np
    subject = sub.split('KPE')[1]
    confound_template = '/media/Data/Lab_Projects/KPE_PTSD_Project/neuroimaging/KPE_BIDS/derivatives/fmriprep/sub-{sub}/ses-{ses}/func/sub-{sub}_ses-{ses}_task-Memory_desc-confounds_regressors.tsv'
    confoundFile = confound_template.format(sub=subject, ses=ses)
    confound = pd.read_csv(confoundFile,sep="\t", na_values="n/a")
    finalConf = confound[['csf', 'white_matter', 'framewise_displacement', 
                          'a_comp_cor_00', 'a_comp_cor_01',	'a_comp_cor_02', 'a_comp_cor_03', 
                          'a_comp_cor_04', 'a_comp_cor_05', 'trans_x', 'trans_y', 'trans_z', 
                          'rot_x', 'rot_y', 'rot_z']] # can add 'global_signal' also ,,
                          # 
     # change NaN of FD to zero
    finalConf = finalConf.fillna(0)
    return np.array(finalConf)

In [ ]:
# method to generate subject array of timeseries
def pooledTS(sub, ses, confounds, output_dir):
    import os
    import numpy as np
    import nilearn.input_data
    import nilearn.image
    
    event_template = '/media/Data/Lab_Projects/KPE_PTSD_Project/neuroimaging/KPE_BIDS/condition_files/withNumbers/sub-{sub}_ses-{ses}_30sec_window.csv'
    func_template = '/media/Data/Lab_Projects/KPE_PTSD_Project/neuroimaging/KPE_BIDS/derivatives/fmriprep/sub-{sub}/ses-{ses}/func/sub-{sub}_ses-{ses}_task-Memory_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'
    # set mask params
    mask_params = {
               'detrend': True, 'standardize': True,
               'high_pass': 0.01, 'low_pass': 0.1, 't_r': 1,
               'smoothing_fwhm': 4,
                'verbose': 5}

    amg_file = '/media/Data/work/KPE_ROI/amygdala_association-test_z_FDR_0.01.nii.gz'
    amg_file = nilearn.image.math_img("a>=25", a=amg_file)
    hipp_mask = '/media/Data/work/KPE_ROI/hippocampus_association-test_z_FDR_0.01.nii.gz'
    hipp_mask = nilearn.image.math_img("a>=15", a=hipp_mask)
    vmpfc_mask = '/media/Data/work/RCF_or/vmpfc_association-test_z_FDR_0.01.nii.gz'
    vmpfc_mask = nilearn.image.math_img("a>=5", a=vmpfc_mask)

    
    # fit maskers
    masker_amg = nilearn.input_data.NiftiMasker(mask_img= amg_file, **mask_params).fit()
    masker_vmpfc = nilearn.input_data.NiftiMasker(mask_img= vmpfc_mask, **mask_params).fit()
    masker_hipp = nilearn.input_data.NiftiMasker(mask_img= hipp_mask, **mask_params).fit()
    # set output dir
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    duration = 120 #set duration of event in seconds 
    #sub_tsAmg = []
    #sub_tsHipp = []
    #sub_tsvmPFC = []
    #for sub in subject_list:
    subject = sub.split('KPE')[1]

    # load the npy file (timeseries)
    tsAmg = masker_amg.transform(func_template.format(sub=subject, ses=ses), confounds)
    tsHipp = masker_hipp.transform(func_template.format(sub=subject, ses=ses),confounds)
    tsvmPFC = masker_vmpfc.transform(func_template.format(sub=subject, ses=ses),confounds)
    event = event_template.format(sub=subject, ses=ses)
    events = pd.read_csv(event, sep='\t')
    onset = int(events.onset[events.trial_type_30=='trauma1_0']) # take onset of trauma first script
    tsAmg_script = tsAmg[onset:onset+duration, :]
    tsHipp_script = tsHipp[onset:onset+duration, :]
    tsvmPFC_script = tsvmPFC[onset:onset+duration, :]
    tsAmg_mean = np.mean(tsAmg_script, axis=1)
    tsHipp_mean = np.mean(tsHipp_script, axis=1)
    tsvmPFC_mean = np.mean(tsvmPFC_script, axis=1)
    np.save(output_dir + 'sub-' + subject + 'amygdalaMeanTS' + 'ses-' + ses, tsAmg_mean)
    np.save(output_dir + 'sub-' + subject + 'hippoMeanTS' + 'ses-' + ses, tsHipp_mean)
    np.save(output_dir + 'sub-' + subject + 'vmPFCMeanTS' + 'ses-' + ses, tsvmPFC_mean)
#     sub_tsAmg.append(tsAmg_mean)
#     sub_tsHipp.append(tsHipp_mean)
#     sub_tsvmPFC.append(tsvmPFC_mean)
    return tsAmg_mean, tsHipp_mean , tsvmPFC_mean

In [ ]:
## condition labels (ketamine , midazolam)
# read file
medication_cond = pd.read_csv('/home/or/kpe_task_analysis/task_based_analysis/kpe_sub_condition.csv')
subject_list = np.array(medication_cond.scr_id)
condition_label = np.array(medication_cond.med_cond)



In [ ]:
subject_list = subject_list[0:24] # removing 1578 
condition_label = condition_label[0:24]
subject_list

In [ ]:
infosource = pe.Node(util.IdentityInterface(fields=['subject_id'
                                            ],
                                    ),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]

removeVars = pe.Node(util.Function(
                    input_names=['sub','ses'],
                    output_names=['finalConf'],
                                     function=removeVars),
                   name="removeVars")

removeVars.inputs.ses = ses

runTimeSeries = pe.Node(util.Function(
                    input_names=['sub','ses','confounds', 'output_dir'],
                    output_names=['tsAmg_mean', 'tsHipp_mean', 'tsvmPFC_mean'],
                                     function=pooledTS),
                   name="runTimeSeries")


runTimeSeries.inputs.output_dir = output_dir
runTimeSeries.inputs.ses = ses


In [ ]:
amg_file

In [ ]:
runTS = pe.Workflow(name='hippAmgTS', base_dir = output_dir)
runTS.connect([
    (infosource, removeVars, [('subject_id', 'sub')]),
    (infosource, runTimeSeries, [('subject_id', 'sub')]),
    (removeVars, runTimeSeries, [('finalConf', 'confounds')])
])

In [ ]:
runTS.run('MultiProc', plugin_args={'n_procs': 10})

In [ ]:
#subject_list = ['KPE008']
subAmg, subHipp, subVMPFC = pooledTS(subject_list, '1')

In [ ]:
subAmg_2, subHipp_2 , subVMPFC_2= pooledTS(subject_list, '2')

In [ ]:
plt.plot(subAmg[1])
plt.plot(subHipp[1])
plt.plot(subVMPFC[1])

In [ ]:
np.array(subAmg_2).shape

In [ ]:
#add both to same array
both1 = np.dstack([subAmg, subHipp, subVMPFC])
both1.shape
both2 = np.dstack([subAmg_2, subHipp_2, subVMPFC_2])
both2.shape


In [ ]:
from nilearn import connectome
connectome = connectome.ConnectivityMeasure(
    kind='correlation', vectorize=False)

mat_ses1 = connectome.fit_transform(both1)

In [ ]:
sns.heatmap(np.mean(mat_ses1, axis=0), annot=True, cmap='coolwarm')

In [ ]:
mat_ses2 = connectome.fit_transform(both2)
sns.heatmap(np.mean(mat_ses2, axis=0), annot=True, cmap="coolwarm")

In [ ]:
ket1 = mat_ses1[condition_label==1]
mid1 = mat_ses1[condition_label==0]
ket2 = mat_ses2[condition_label==1]
mid2 = mat_ses2[condition_label==0]
sns.heatmap(np.mean(ket2, axis=0), cmap='coolwarm', annot=True)
plt.show()
sns.heatmap(np.mean(mid2, axis=0), cmap='coolwarm', annot=True)

In [ ]:
t, p =scipy.stats.ttest_ind(ket2, mid2)

In [ ]:
p

In [ ]:
ket2_1 = np.subtract(ket2, ket1)
mid2_1 = np.subtract(mid2, mid1)

In [ ]:
t, p =scipy.stats.ttest_ind(ket2_1, mid2_1)
p

In [ ]:
sns.heatmap(t, cmap='coolwarm', annot=True)

In [ ]:
mat2_1 = np.subtract(mat_ses2, mat_ses1)
dfDelta = pd.DataFrame({'scr_id':subject_list, 'group':condition_label,
                        'amg_hippo': mat2_1[:,0,1],
                        'amg_vpmfc': mat2_1[:,0,2]})

In [ ]:
sns.boxplot( y= 'amg_vpmfc',x= 'group', data= dfDelta, saturation=0.2)
sns.stripplot( y= 'amg_vpmfc',x= 'group', data= dfDelta)
scipy.stats.ttest_ind(dfDelta.amg_vpmfc[dfDelta.group==1], 
                      dfDelta.amg_vpmfc[dfDelta.group==0])

In [ ]:
import pymc3 as pm
from pymc3.glm import GLM

with pm.Model() as model_glm:
    GLM.from_formula('amg_vpmfc ~ group', dfDelta)
    trace = pm.sample(draws=4000, tune=3000)

In [ ]:
pm.summary(trace, credible_interval=.95).round(2)

In [ ]:
ci = np.quantile(trace.group, [.025,.975]) # take credible intervals from the trace

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(4, 5),gridspec_kw={'width_ratios': [1, .2],
                                                        'wspace':.1})
g1 = sns.stripplot(y= 'amg_vpmfc', x='group', data=dfDelta, size = 8, ax=ax1)
sns.boxplot(y= 'amg_vpmfc', x='group', data=dfDelta,  ax=ax1,
            boxprops=dict(alpha=.3))
g2 = sns.distplot(trace['group'], ax = ax2, vertical=True)
ax2.vlines(x=0.2,ymin=ci[0], ymax=ci[1], color='black', 
           linewidth = 1.5, linestyle = "-")

#g3.set_ylim(-.7, .7)
ax1.set_ylim(-.7,.7)
ax2.set_ylim(-.7,.7)
ax2.set_ylabel("Difference between groups", fontsize=14) 
ax2.yaxis.set_label_position("right")
ax2.yaxis.tick_right()
ax2.set_xticks([])
ax1.set_ylabel("Amg-vmPFC change in connectivity", fontsize=14)
ax1.set_xlabel("Group", fontsize=14)
ax1.set_xticklabels(['Midazolam', 'Ketamine'])
fig.savefig("amgvmPFC.png", dpi=300,  bbox_inches='tight')

In [ ]:
# only first 30sec
both1_30 = both1[:, 0:120, :]
both2_30 = both2[:, 0:120, :]

mat_ses1_30 = connectome.fit_transform(both1_30)
mat_ses2_30 = connectome.fit_transform(both2_30)

In [ ]:
ket1 = mat_ses1_30[condition_label==1]
mid1 = mat_ses1_30[condition_label==0]
ket2 = mat_ses2_30[condition_label==1]
mid2 = mat_ses2_30[condition_label==0]

ket2_1 = np.subtract(ket2, ket1)
mid2_1 = np.subtract(mid2, mid1)

In [ ]:
t, p =scipy.stats.ttest_ind(ket2, mid2)
p

In [ ]:
## test for different time frames (30,60,90,120) sec
ses = '2'
duration = 60
for sub in subject_list:
        subject = sub.split('KPE')[1]
        amgTS = np.load(output_dir + 'sub-' + subject + 'amygdalaMeanTS' + 'ses-' + ses + '.npy')
        hippTS = np.load(output_dir + 'sub-' + subject + 'hippoMeanTS' + 'ses-' + ses + '.npy')
        vmpfcTS = np.load(output_dir + 'sub-' + subject + 'vmPFCMeanTS' + 'ses-' + ses + '.npy')
        